
# PRIDE- Parkinson's Research through Integrative Data Experiments

## Code for creating Gephi Graph

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import os
import gzip
import pylab as plt
import json
import os


## Data Preprocessing
Worked with files downloaded from:
http://ctdbase.org/downloads/;jsessionid=74BC65C36365CF4BCC193C670F9C670E#cd

### Prepare nodes table

In [3]:

#Gene vocabulary
#GeneSymbol
#GeneName
#GeneID (NCBI Gene identifier)
#AltGeneIDs (alternative NCBI Gene identifiers; '|'-delimited list)
#Synonyms ('|'-delimited list)
#BioGRIDIDs ('|'-delimited list)
#PharmGKBIDs ('|'-delimited list)
#UniprotIDs ('|'-delimited list)

genes = pd.read_csv('CTD_genes.csv', skiprows=range(1, 29),
                      names = ['GeneSymbol',
                               'GeneName',
                               'GeneID',
                               'AltGeneIDs',
                               'Synonyms',
                               'BioGRIDIDs',
                               'PharmGKBIDs',
                               'UniprotIDs'])
genes = genes[1:]
genes.drop(['AltGeneIDs','Synonyms','BioGRIDIDs',
                           'PharmGKBIDs','UniprotIDs'], axis = 1, inplace = True)

In [4]:
genes.shape

(532824, 3)

In [5]:
genes.head()

,GeneSymbol,GeneName,GeneID
1,03B03F,"DNA segment, 03B03F (Research Genetics)",27777.0
2,03B03R,"DNA segment, 03B03R (Research Genetics)",27778.0
3,03.MMHAP34FRA.SEQ,"DNA segment, 03.MMHAP34FRA.seq",53288.0
4,064YA,NaN,5658107.0
5,102G4T7,"DNA segment, 102g4T7",56573.0


In [6]:
genes.isnull().sum()

GeneSymbol       1
GeneName      2431
GeneID           0
dtype: int64

In [7]:
genes.dropna().shape

(530392, 3)

In [8]:
# prepare nodes csv for gephi
gene_nodes = genes[['GeneSymbol','GeneName']]
#gene_nodes.columns = ['Id', 'Label']

In [ ]:
gene_nodes.to_csv("gene_nodes.csv",index=False)

In [9]:
#Chemical vocabulary
#Fields:

#    ChemicalName
#   ChemicalID (MeSH identifier)
#   CasRN (CAS Registry Number, if available)
#   Definition
#    ParentIDs (identifiers of the parent terms; '|'-delimited list)
#    TreeNumbers (identifiers of the chemical's nodes; '|'-delimited list)
#    ParentTreeNumbers (identifiers of the parent nodes; '|'-delimited list)
#    Synonyms ('|'-delimited list)

chem = pd.read_csv('CTD_chemicals.csv', skiprows=range(1, 27),header = 1)


In [10]:
chem = chem.drop([0])
chem.head()

,# ChemicalName,ChemicalID,CasRN,Definition,ParentIDs,TreeNumbers,ParentTreeNumbers,Synonyms
1,(0.017ferrocene)amylose,MESH:C089250,NaN,NaN,MESH:D000075163|MESH:D000688|MESH:D005296,D01.490.200/C089250|D02.691.657/C089250|D09.30...,D01.490.200|D02.691.657|D09.301.915.361|D09.69...,(0.017 ferrocene)amylose
2,001-C8-NBD,MESH:C114385,NaN,NaN,MESH:D009842|MESH:D010069,D03.383.129.462.580/C114385|D12.644.456/C114385,D03.383.129.462.580|D12.644.456,001 C8 NBD|H-MeTyr-Arg-MeArg-D-Leu-NH(CH2)8NH-...
3,001-C8 oligopeptide,MESH:C114386,NaN,NaN,MESH:D009842,D12.644.456/C114386,D12.644.456,001 C8 oligopeptide|H-MeTyr-Arg-MeArg-D-Leu-NH...
4,"0231A , Streptomyces",MESH:C434150,NaN,NaN,MESH:D006576,D03.633.400/C434150,D03.633.400,NaN
5,"0231B, Streptomyces",MESH:C434149,NaN,NaN,MESH:D006576,D03.633.400/C434149,D03.633.400,NaN


In [11]:
chem.shape

(173886, 8)

In [12]:
chem.isnull().sum()

# ChemicalName            0
ChemicalID                0
CasRN                117488
Definition           167999
ParentIDs                 1
TreeNumbers               0
ParentTreeNumbers         1
Synonyms              60965
dtype: int64

In [13]:
# prepare nodes csv for gephi
chem_nodes = chem.iloc[:,:2]
chem_nodes.columns = ['Label', 'Id']
chem_nodes.head()

,Label,Id
1,(0.017ferrocene)amylose,MESH:C089250
2,001-C8-NBD,MESH:C114385
3,001-C8 oligopeptide,MESH:C114386
4,"0231A , Streptomyces",MESH:C434150
5,"0231B, Streptomyces",MESH:C434149


In [14]:
# Disease vocabulary (MEDIC)
#Fields (non-OBO):

#    DiseaseName
#    DiseaseID (MeSH or OMIM identifier)
#    Definition
#    AltDiseaseIDs (alternative identifiers; '|'-delimited list)
#    ParentIDs (identifiers of the parent terms; '|'-delimited list)
#    TreeNumbers (identifiers of the disease's nodes; '|'-delimited list)
#    ParentTreeNumbers (identifiers of the parent nodes; '|'-delimited list)
#    Synonyms ('|'-delimited list)
#    SlimMappings (MEDIC-Slim mappings; '|'-delimited list)

disease = pd.read_csv('CTD_diseases.csv', skiprows=range(1, 27),header=1)
disease = disease.drop([0])
disease.head()

,# DiseaseName,DiseaseID,AltDiseaseIDs,Definition,ParentIDs,TreeNumbers,ParentTreeNumbers,Synonyms,SlimMappings
1,10p Deletion Syndrome (Partial),MESH:C538288,NaN,NaN,MESH:D002872|MESH:D025063,C16.131.260/C538288|C16.320.180/C538288|C23.55...,C16.131.260|C16.320.180|C23.550.210.050.500.500,"Chromosome 10, 10p- Partial|Chromosome 10, mon...",Congenital abnormality|Genetic disease (inborn...
2,13q deletion syndrome,MESH:C535484,NaN,NaN,MESH:D002872|MESH:D025063,C16.131.260/C535484|C16.320.180/C535484|C23.55...,C16.131.260|C16.320.180|C23.550.210.050.500.500,Chromosome 13q deletion|Chromosome 13q deletio...,Congenital abnormality|Genetic disease (inborn...
3,15q24 Microdeletion,MESH:C579849,DO:DOID:0060395,NaN,MESH:D002872|MESH:D008607|MESH:D025063,C10.597.606.360/C579849|C16.131.260/C579849|C1...,C10.597.606.360|C16.131.260|C16.320.180|C23.55...,15q24 Deletion|15q24 Microdeletion Syndrome|In...,Congenital abnormality|Genetic disease (inborn...
4,16p11.2 Deletion Syndrome,MESH:C579850,NaN,NaN,MESH:D001321|MESH:D002872|MESH:D008607|MESH:D0...,C10.597.606.360/C579850|C16.131.260/C579850|C1...,C10.597.606.360|C16.131.260|C16.320.180|C23.55...,NaN,Congenital abnormality|Genetic disease (inborn...
5,"17,20-Lyase Deficiency, Isolated",MESH:C567076,NaN,NaN,MESH:D000312,C12.706.316.090.500/C567076|C13.351.875.253.09...,C12.706.316.090.500|C13.351.875.253.090.500|C1...,"17-Alpha-Hydroxylase-17,20-Lyase Deficiency, C...",Congenital abnormality|Endocrine system diseas...


In [15]:
disease.isnull().sum()

# DiseaseName           0
DiseaseID               0
AltDiseaseIDs        6646
Definition           8083
ParentIDs               1
TreeNumbers             0
ParentTreeNumbers       1
Synonyms             2042
SlimMappings           57
dtype: int64

In [16]:
# prepare nodes csv for gephi
disease_nodes = disease.iloc[:,:2]
disease_nodes.columns = ['Label', 'Id']
disease_nodes.head()

,Label,Id
1,10p Deletion Syndrome (Partial),MESH:C538288
2,13q deletion syndrome,MESH:C535484
3,15q24 Microdeletion,MESH:C579849
4,16p11.2 Deletion Syndrome,MESH:C579850
5,"17,20-Lyase Deficiency, Isolated",MESH:C567076


In [17]:
nodes = pd.concat([gene_nodes,chem_nodes,disease_nodes])
nodes.to_csv("nodes.csv",index=False)

In [18]:
nodes.head()

,GeneName,GeneSymbol,Id,Label
1,"DNA segment, 03B03F (Research Genetics)",03B03F,NaN,NaN
2,"DNA segment, 03B03R (Research Genetics)",03B03R,NaN,NaN
3,"DNA segment, 03.MMHAP34FRA.seq",03.MMHAP34FRA.SEQ,NaN,NaN
4,NaN,064YA,NaN,NaN
5,"DNA segment, 102g4T7",102G4T7,NaN,NaN


### Prepare edge table

In [ ]:
#Gene–disease associations
# Fields:

#     GeneSymbol
#     GeneID (NCBI Gene identifier)
#     DiseaseName
#     DiseaseID (MeSH or OMIM identifier)
#     DirectEvidence ('|'-delimited list)
#     InferenceChemicalName
#     InferenceScore
#     OmimIDs ('|'-delimited list)
#     PubMedIDs ('|'-delimited list)

genedisease = pd.read_csv('CTD_genes_diseases.csv', header = 1,skiprows=range(1, 27))
genedisease = genedisease[1:]
#genedisease.drop(['casrn','pubmedids', 'Unnamed: 13','comentionedterms','organism','interaction','anatomyterms','inferencegenesymbols','organismid'], axis = 1, inplace = True)
genedisease.head()

In [20]:
#Chemical–gene interaction
#Fields:
#ChemicalName
#ChemicalID (MeSH identifier)
#CasRN (CAS Registry Number, if available)
#GeneSymbol
#GeneID (NCBI Gene identifier)
#GeneForms ('|'-delimited list)
#Organism (scientific name)
#OrganismID (NCBI Taxonomy identifier)
#Interaction
#InteractionActions ('|'-delimited list)
#PubMedIDs ('|'-delimited list)

chemgene = pd.read_csv('CTD_chem_gene_ixns.csv', skiprows=range(1, 29), 
                       names = ['ChemicalName',
                               'ChemicalID',
                               'CasRN',
                               'GeneSymbol',
                               'GeneID',
                               'GeneForms',
                               'Organism',
                               'OrganismID',
                               'Interaction',
                               'InteractionActions',
                               'PubMedIDs'])
chemgene = chemgene[1:]
chemgene.drop(['CasRN','PubMedIDs','Organism','OrganismID','Interaction','GeneForms'], axis = 1, inplace = True)

In [21]:
chemgene.head()

,ChemicalName,ChemicalID,GeneSymbol,GeneID,InteractionActions
1,10074-G5,C534883,AR,367.0,affects^reaction|increases^expression
2,10074-G5,C534883,AR,367.0,decreases^reaction|increases^expression
3,10074-G5,C534883,AR,367.0,decreases^expression
4,10074-G5,C534883,AR,367.0,decreases^expression
5,10074-G5,C534883,EPHB2,2048.0,decreases^reaction|increases^expression


In [22]:
chemgene.shape

(2053659, 5)

In [23]:
chemgene.isnull().sum()

ChemicalName          0
ChemicalID            0
GeneSymbol            2
GeneID                0
InteractionActions    0
dtype: int64

In [24]:
chemgene = chemgene.drop_duplicates()

In [25]:
# prepare as gephi edges
chemgene_edge = chemgene[["ChemicalID","GeneID","InteractionActions"]]
chemgene_edge.columns = ["Source", "Target", "InteractionActions"]
chemgene_edge.head()

,Source,Target,InteractionActions
1,C534883,367.0,affects^reaction|increases^expression
2,C534883,367.0,decreases^reaction|increases^expression
3,C534883,367.0,decreases^expression
5,C534883,2048.0,decreases^reaction|increases^expression
7,C534883,4149.0,affects^binding|affects^folding|decreases^acti...


In [ ]:
chemgene_edge.to_csv("chemgene_edge.csv", index=False)

In [26]:
#Chemical disease associations
#Fields:
#ChemicalName
#ChemicalID (MeSH identifier)
#CasRN (CAS Registry Number, if available)
#DiseaseName
#DiseaseID (MeSH or OMIM identifier)
#DirectEvidence ('|'-delimited list)
#InferenceGeneSymbol
#InferenceScore
#OmimIDs ('|'-delimited list)
#PubMedIDs ('|'-delimited list)

chemdisease = pd.read_csv('CTD_chemicals_diseases.csv',skiprows=range(1, 29), 
                       names = ['ChemicalName',
                               'ChemicalID',
                               'CasRN',
                               'DiseaseName',
                               'DiseaseID',
                               'DirectEvidence',
                               'InferenceGeneSymbol',
                               'InferenceScore',
                               'OmimIDs',
                               'PubMedIDs'])
chemdisease = chemdisease[1:]
chemdisease.head()

,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DirectEvidence,InferenceGeneSymbol,InferenceScore,OmimIDs,PubMedIDs
1,06-Paris-LA-66 protocol,C046983,NaN,Precursor Cell Lymphoblastic Leukemia-Lymphoma,MESH:D054198,therapeutic,NaN,NaN,NaN,4519131
2,10074-G5,C534883,NaN,Adenocarcinoma,MESH:D000230,NaN,MYC,4.08,NaN,26432044
3,10074-G5,C534883,NaN,Adenocarcinoma of Lung,MESH:D000077192,NaN,MYC,4.32,NaN,26656844|27602772
4,10074-G5,C534883,NaN,Alopecia,MESH:D000505,NaN,AR,4.51,NaN,15902657
5,10074-G5,C534883,NaN,Androgen-Insensitivity Syndrome,MESH:D013734,NaN,AR,6.86,300068|312300,1303262|8281139


In [27]:
chemdisease.isnull().sum()

ChemicalName                 0
ChemicalID                   0
CasRN                  2148292
DiseaseName                  0
DiseaseID                    0
DirectEvidence         6968776
InferenceGeneSymbol     100727
InferenceScore          100727
OmimIDs                6523668
PubMedIDs               416707
dtype: int64

In [28]:
chemdisease = chemdisease.drop_duplicates()

In [29]:
chemdisease.shape

(7069503, 10)

In [30]:
# prepare as gephi edges
chemdisease_edge = chemdisease[["ChemicalID","DiseaseID","DirectEvidence"]]
chemdisease_edge.columns = ["Source", "Target", "DirectEvidence"]
chemdisease_edge.head()

,Source,Target,DirectEvidence
1,C046983,MESH:D054198,therapeutic
2,C534883,MESH:D000230,NaN
3,C534883,MESH:D000077192,NaN
4,C534883,MESH:D000505,NaN
5,C534883,MESH:D013734,NaN


In [ ]:
chemdisease_edge.to_csv("chemdisease_edge.csv",index=False)

### To limit the size of database to fit computer memory, limit to Parkinson related genes

In [31]:
# download from CTD website with searching criteria
parkinson_gene = pd.read_csv("CTD_D010300_genes_20201106165349.csv")

In [32]:
parkinson_gene.head()

,Gene Symbol,Gene ID,Disease Name,Disease ID,Direct Evidence,Inference Network,Inference Score,Reference Count
0,SOD2,6648,Parkinson Disease,MESH:D010300,marker/mechanism,"1-Methyl-4-phenyl-1,2,3,6-tetrahydropyridine|1...",123.93,365
1,BDNF,627,Parkinson Disease,MESH:D010300,marker/mechanism,"1-Methyl-4-phenyl-1,2,3,6-tetrahydropyridine|1...",121.54,167
2,SNCA,6622,Parkinson Disease,MESH:D010300,marker/mechanism,"1,2,3,4-tetrahydro-1-(phenylmethyl)isoquinolin...",118.06,189
3,HMOX1,3162,Parkinson Disease,MESH:D010300,marker/mechanism,"1-Methyl-4-phenyl-1,2,3,6-tetrahydropyridine|1...",111.51,161
4,GSTM1,2944,Parkinson Disease,MESH:D010300,marker/mechanism,1-Methyl-4-phenylpyridinium|Air Pollutants|Asp...,104.27,88


In [33]:
len(parkinson_gene)

24135

In [34]:
parkinson_gene.isnull().sum()

Gene Symbol              0
Gene ID                  0
Disease Name             0
Disease ID               0
Direct Evidence      24002
Inference Network       45
Inference Score         45
Reference Count          0
dtype: int64

In [35]:
len(parkinson_gene['Gene ID'].unique())

23990

In [36]:
parkinson_gene = parkinson_gene.drop_duplicates(subset=['Gene ID'], keep='first')

In [37]:
parkinson_gene.shape

(23990, 8)

#### Prepare parkinson_gene_edge

In [38]:

parkinson_gene_edge = parkinson_gene[['Gene ID', 'Disease ID', 'Direct Evidence']]
parkinson_gene_edge.columns = ["Source", "Target", "Direct Evidence"]
parkinson_gene_edge.head()

,Source,Target,Direct Evidence
0,6648,MESH:D010300,marker/mechanism
1,627,MESH:D010300,marker/mechanism
2,6622,MESH:D010300,marker/mechanism
3,3162,MESH:D010300,marker/mechanism
4,2944,MESH:D010300,marker/mechanism


In [39]:
genes.head()

,GeneSymbol,GeneName,GeneID
1,03B03F,"DNA segment, 03B03F (Research Genetics)",27777.0
2,03B03R,"DNA segment, 03B03R (Research Genetics)",27778.0
3,03.MMHAP34FRA.SEQ,"DNA segment, 03.MMHAP34FRA.seq",53288.0
4,064YA,NaN,5658107.0
5,102G4T7,"DNA segment, 102g4T7",56573.0


In [ ]:
parkinson_gene_edge.to_csv('parkinson_gene_edge.csv', index=False)

In [40]:
parkinson_gene['Direct Evidence'].unique()

array(['marker/mechanism', 'marker/mechanism|therapeutic', 'therapeutic',
       nan], dtype=object)

#### Find chemicals related with genes (which has interaction with parkinsons)

In [41]:
parkinson_gene_chem = parkinson_gene.join(chemgene, on='Gene ID', how='inner', sort=False)

In [42]:
parkinson_gene_chem.shape

(19026, 13)

In [43]:
parkinson_gene_chem.head()

,Gene Symbol,Gene ID,Disease Name,Disease ID,Direct Evidence,Inference Network,Inference Score,Reference Count,ChemicalName,ChemicalID,GeneSymbol,GeneID,InteractionActions
0,SOD2,6648,Parkinson Disease,MESH:D010300,marker/mechanism,"1-Methyl-4-phenyl-1,2,3,6-tetrahydropyridine|1...",123.93,365,"1,2-Dimethylhydrazine",D019813,CASP4,837.0,affects^cotreatment|decreases^expression
2,SNCA,6622,Parkinson Disease,MESH:D010300,marker/mechanism,"1,2,3,4-tetrahydro-1-(phenylmethyl)isoquinolin...",118.06,189,"1,2-Dimethylhydrazine",D019813,CARD14,79092.0,increases^expression
3,HMOX1,3162,Parkinson Disease,MESH:D010300,marker/mechanism,"1-Methyl-4-phenyl-1,2,3,6-tetrahydropyridine|1...",111.51,161,"1,2,5,6-tetrabromocyclooctane",C540596,VCL,7414.0,decreases^expression
4,GSTM1,2944,Parkinson Disease,MESH:D010300,marker/mechanism,1-Methyl-4-phenylpyridinium|Air Pollutants|Asp...,104.27,88,"1,2,5,6-dibenzanthracene",C026486,ZCCHC2,54877.0,increases^expression
5,TH,7054,Parkinson Disease,MESH:D010300,marker/mechanism,1-(3-(2-(1-benzothiophen-5-yl) ethoxy) propyl)...,102.60,347,"1,2-Dimethylhydrazine",D019813,CEBPD,1052.0,affects^cotreatment|decreases^expression


In [44]:
# how many genes related to parkinson
len(parkinson_gene_chem['Gene ID'].unique())

19026

In [45]:
# how many chemicals  
len(parkinson_gene_chem['ChemicalID'].unique())

973

In [46]:
parkinsonChem_nodes = parkinson_gene_chem[['ChemicalName','ChemicalID']]
parkinsonChem_nodes.columns = ['Label', 'Id']
parkinsonChem_nodes.to_csv('parkinsonChem_nodes.csv',index=False)

In [47]:
parkinsonGene_nodes = parkinson_gene_chem[['GeneSymbol','GeneID']]
parkinsonGene_nodes.columns = ['Label', 'Id']
parkinsonGene_nodes.to_csv('parkinsonGene_nodes.csv',index=False)

In [48]:
parkinsonGeneChem_edges = parkinson_gene_chem[['GeneID','ChemicalID','InteractionActions']]
parkinsonGeneChem_edges.columns = ['Source', 'Target', 'InteractionActions']
parkinsonGeneChem_edges.to_csv('parkinsonGeneChem_edges.csv', index=False)

### Collect drug data from drugbank



In [49]:
drugbank = pd.read_csv("drugBank_v515_targetExtracted.csv")

In [50]:
drugbank.head()

,Unnamed: 0,dg_id,dg_name,dg_synm,dg_kingdom,dg_superclass,dg_interactions,dg_ext_id,dg_pathways,target_name,target_uniprot,target_gene_name,action,cell_loc
0,0,DB00001,Lepirudin,Hirudin variant-1;Lepirudin recombinant,Organic Compounds,Organic Acids,DB06605;DB06695;DB01254;DB01609;DB01586;DB0212...,Drugs Product Database (DPD):11916;PubChem Sub...,Lepirudin Action Pathway,Prothrombin,P00734,F2,inhibitor,Secreted
1,1,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00012;DB00016;DB08894;DB09107;DB00255;DB0026...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Epidermal growth factor receptor,P00533,EGFR,antagonist,Cell membrane
2,2,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00012;DB00016;DB08894;DB09107;DB00255;DB0026...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Low affinity immunoglobulin gamma Fc region re...,O75015,FCGR3B,NaN,Cell membrane
3,3,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00012;DB00016;DB08894;DB09107;DB00255;DB0026...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Complement C1q subcomponent subunit A,P02745,C1QA,NaN,Secreted
4,4,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00012;DB00016;DB08894;DB09107;DB00255;DB0026...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Complement C1q subcomponent subunit B,P02746,C1QB,NaN,Secreted


In [51]:
len(drugbank['target_gene_name'])

14441

In [52]:
a = set(drugbank['target_gene_name'])

In [55]:
b = set(genes['GeneSymbol'])

In [56]:
len(a&b)

2733

In [57]:
commonn_gene = (a&b)

In [58]:
# find all gene data which are also recorded in drugbank
drug_common_gene = drugbank[drugbank['target_gene_name'].isin(commonn_gene)]
drug_common_gene.head()

,Unnamed: 0,dg_id,dg_name,dg_synm,dg_kingdom,dg_superclass,dg_interactions,dg_ext_id,dg_pathways,target_name,target_uniprot,target_gene_name,action,cell_loc
0,0,DB00001,Lepirudin,Hirudin variant-1;Lepirudin recombinant,Organic Compounds,Organic Acids,DB06605;DB06695;DB01254;DB01609;DB01586;DB0212...,Drugs Product Database (DPD):11916;PubChem Sub...,Lepirudin Action Pathway,Prothrombin,P00734,F2,inhibitor,Secreted
1,1,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00012;DB00016;DB08894;DB09107;DB00255;DB0026...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Epidermal growth factor receptor,P00533,EGFR,antagonist,Cell membrane
2,2,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00012;DB00016;DB08894;DB09107;DB00255;DB0026...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Low affinity immunoglobulin gamma Fc region re...,O75015,FCGR3B,NaN,Cell membrane
3,3,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00012;DB00016;DB08894;DB09107;DB00255;DB0026...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Complement C1q subcomponent subunit A,P02745,C1QA,NaN,Secreted
4,4,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00012;DB00016;DB08894;DB09107;DB00255;DB0026...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Complement C1q subcomponent subunit B,P02746,C1QB,NaN,Secreted


In [59]:
drug_common_gene_nodes = drug_common_gene[['dg_id','dg_name']]
drug_common_gene_nodes.columns = ['Id','Label']
drug_common_gene_nodes['Type'] = 'Drug'
drug_common_gene_nodes.to_csv('drug_common_gene_nodes.csv',index=False)

/N/soft/rhel7/anaconda/python3.6/4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [62]:
drug_common_gene_edges = drug_common_gene[['dg_id','target_gene_name']]
drug_common_gene_edges.columns=['Source','Target']
drug_common_gene_edges['Interaction'] = 'Treat'
drug_common_gene_edges.columns=['Source','GeneSymbol','Interaction']
drug_common_gene_edges = drug_common_gene_edges.merge(genes,on='GeneSymbol',how='left')
drug_common_gene_edges.columns = ['Source','GeneSymbol','Interaction', 'GeneName', 'Target']
drug_common_gene_edges.to_csv('drug_common_gene_edges.csv', index=False)

/N/soft/rhel7/anaconda/python3.6/4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
drug_common_gene_edges.head()

,Source,GeneSymbol,Interaction,GeneName,Target
0,DB00001,F2,Treat,"coagulation factor II, thrombin",2147.0
1,DB00002,EGFR,Treat,epidermal growth factor receptor,1956.0
2,DB00002,FCGR3B,Treat,Fc fragment of IgG receptor IIIb,2215.0
3,DB00002,C1QA,Treat,complement C1q A chain,712.0
4,DB00002,C1QB,Treat,complement C1q B chain,713.0


## Import nodes and edges csv files in Gephi, create graphs. Sample graph shown below:

![Sample Graph]("img/screenshot_092437")

In [65]:
from IPython.display import Image
Image("img/screenshot_092437")